In [104]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

/Users/divyam/Dev/Data_Science/DSF_Project/AcademicPaperRanking
['AcademicPaperRanking', 'acm.csv']


In [116]:
def create_csv_file(txt_file, csv_file):
    """
    converts a text file into csv_file.
    function expects the column values and given starting expression in text for its identification to be "same".
    """
    with open(txt_file,"r") as input_file, open(csv_file, "w") as csv_file:
        column = {"PaperTitle":"", "Authors":"", "Year":"","Publication_Venue":"", "Index_Id":"", "References_Id":"","Abstract":""}
        writer = csv.writer(csv_file)
        writer.writerow(column.keys())
        for l in input_file.readlines():
            l = l.strip()
            if l.startswith("#*"):
                column["PaperTitle"] = l[2:]
            elif l.startswith("#@"):
                column["Authors"] = l[2:]
            elif l.startswith("#t"):
                column["Year"] = l[2:]
            elif l.startswith("#c"):
                column["Publication_Venue"] = l[2:]
            elif l.startswith("#index"):
                column["Index_Id"] = l[6:]
            elif l.startswith("#%"):
                references = column.get("References_Id","")
                references += ":" + l[2:]
                column["References_Id"] = references
            elif l.startswith("#!"):
                column["Abstract"] = l[2:]
            elif l == "":
                writer.writerow(column.values())
                column = {"PaperTitle":"", "Authors":"", "Year":"","Publication_Venue":"", "Index_Id":"", "References_Id":"","Abstract":""}
            else:
                print("dump of invalid data for {}:{}".format(column["PaperTitle"],l))

In [117]:
# CSV CONVERSION NOT NEEDED AS CSV FILE IS LOADED HERE
# txt_file = "../acm.txt"
# csv_file = "../acm.csv"
# create_csv_file(txt_file, csv_file)

In [118]:
# CSV CONVERSION NOT NEEDED AS CSV FILE IS LOADED HERE
# txt_file = "../dblp.txt"
# csv_file = "../dblp.csv"
# create_csv_file(txt_file, csv_file)

In [119]:
#load dataframe from csv
%timeit
df = pd.read_csv("../acm.csv")

CPU times: user 12 µs, sys: 12 µs, total: 24 µs
Wall time: 1.12 ms


/Users/divyam/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.



In [121]:
#DBLP indexid
%timeit
df2 = pd.read_csv("../dblp.csv")

/Users/divyam/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.



In [122]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3680006 entries, 0 to 3680005
Data columns (total 7 columns):
PaperTitle           object
Authors              object
Year                 int64
Publication_Venue    object
Index_Id             object
References_Id        object
Abstract             float64
dtypes: float64(1), int64(1), object(5)
memory usage: 196.5+ MB


sample dataset

In [125]:
df2.head()

,PaperTitle,Authors,Year,Publication_Venue,Index_Id,References_Id,Abstract
0,Approximate Distance Oracles with Improved Que...,Christian Wulff-Nilsen,2015,Encyclopedia of Algorithms,555036b37cea80f954149ffc,NaN,NaN
1,Subset Sum Algorithm for Bin Packing.,Julián Mestre,2015,Encyclopedia of Algorithms,555036b37cea80f954149ffd,NaN,NaN
2,Minimum Weighted Completion Time.,"V. S. Anil Kumar, Madhav V. Marathe, Srinivasa...",2015,Encyclopedia of Algorithms,555036b37cea80f954149ffe,NaN,NaN
3,Multiple String Matching.,"Maxime Crochemore, Thierry Lecroq",2015,Encyclopedia of Algorithms,555036b37cea80f954149fff,NaN,NaN
4,Fault-Tolerant Connected Dominating Set.,"Donghyun Kim, Wei Wang, Weili Wu, Alade O. Tokuta",2015,Encyclopedia of Algorithms,555036b37cea80f95414a000,NaN,NaN


In [126]:
df.head()

,PaperTitle,Authors,Year,Publication_Venue,Index_Id,References_Id,Abstract
0,MOSFET table look-up models for circuit simula...,NaN,1984.0,"Integration, the VLSI Journal",1,NaN,NaN
1,The verification of the protection mechanisms ...,Virgil D. Gligor,1984.0,International Journal of Parallel Programming,2,NaN,NaN
2,Another view of functional and multivalued dep...,"M. Gyssens, J. Paredaens",1984.0,International Journal of Parallel Programming,3,NaN,NaN
3,Entity-relationship diagrams which are in BCNF,"Sushil Jajodia, Peter A. Ng, Frederick N. Spri...",1984.0,International Journal of Parallel Programming,4,NaN,NaN
4,The computer comes of age,Rene Moreau,1984.0,The computer comes of age,5,NaN,NaN


In [136]:
###### TODO1: check if there are common "research papers" in DBLP and ACM dataset 
###### TODO2: combine the 2 datasets (indexid's format is different)

***count_reference*** represents the number of times particular paper got referred in our dataset

In [131]:
df["count_reference"] = 0

####YEAR

In [132]:
df["Year"] = df["Year"].fillna(0)
df["Year"] = df["Year"].astype(int)

**Rows with NaN year**

In [134]:
df[df["Year"] == 0].shape

(8428, 8)

***Year of publication distribution in the data***

In [ ]:
ydf = df[df["Year"]!=0]
plt.figure(figsize=(8,6))
plt.scatter(range(ydf.shape[0]), np.sort(ydf["Year"].values))
plt.xlabel('index', fontsize=12)
plt.ylabel('Year', fontsize=12)
plt.title("Year Distribution", fontsize=20) 
plt.show()

In [ ]:
df.isna().sum()

In [ ]:
###### TODO3: DATA-CLEAN

In [ ]:
print(df.dtypes)

In [17]:
df["Index_Id"].unique()
print(any(df['Index_Id'].duplicated())) 
print(len(df['Index_Id'].unique()))
print(len(df.index))

True
2385057
2385066


> Index_Id = 2381731 is duplicated 9 times , It points to 7 different papers and has 82080 references.

In [51]:
df[df.duplicated(subset="Index_Id")]

,PaperTitle,Authors,Year,Publication_Venue,Index_Id,References_Id,Abstract,count_reference
2111180,The online revolution: education for everyone,"Andrew Ng, Daphne Koller",2013,Proceedings of the 19th ACM SIGKDD internation...,2381731,NaN,"In 2011, Stanford University offered three onl...",0
2112019,Predicting the present with search engine data,Hal Varian,2013,Proceedings of the 19th ACM SIGKDD internation...,2381731,NaN,Many businesses now have almost real time data...,0
2112020,Predicting the present with search engine data,Hal Varian,2013,Proceedings of the 19th ACM SIGKDD internation...,2381731,NaN,Many businesses now have almost real time data...,0
2112021,Predicting the present with search engine data,Hal Varian,2013,Proceedings of the 19th ACM SIGKDD internation...,2381731,NaN,Many businesses now have almost real time data...,0
2134028,The DataPaper: living in the virtual world,"Mark Green, Chris Shaw",1990,Graphics Interface 1990,2381731,NaN,Virtual reality user interfaces are a new type...,0
2134993,An efficient diagnosis algorithm for inconsist...,"A. Felfernig, M. Schubert, C. Zehentner",2012,AI EDAM,2381731,NaN,Constraint sets can become inconsistent in dif...,0
2134994,Evaluating Design Alternatives for Feature Rec...,"Monika Mandl, Alexander Felfernig, Juha Tiihonen",2011,Graz,2381731,NaN,Product configuration systems are an important...,0
2381694,gStore: answering SPARQL queries via subgraph ...,"Lei Zou, Jinghui Mo, Lei Chen, M. Tamer Özsu, ...",2011,PVLDB,2381731,NaN,"Due to the increasing use of RDF data, efficie...",0
2381730,Effects of Terms Recognition Mistakes on Reque...,Mohamed Nazih Omri,2012,IJIRR: International Journal of Information Re...,2381731,NaN,In this work the author proposes to describe a...,0


In [25]:
df.Index_Id.describe()

count    2.385066e+06
mean     1.192534e+06
std      6.885107e+05
min      1.000000e+00
25%      5.962672e+05
50%      1.192534e+06
75%      1.788800e+06
max      2.385066e+06
Name: Index_Id, dtype: float64

In [ ]:
reference_count = {}
for references in df[df["References_Id"].notnull()]["References_Id"]:
    ref_list = map(int,references.strip(":").split(":"))
    for ref in ref_list:
        if ref in reference_count:
            reference_count[ref] += 1
        else:
            reference_count[ref] = 1
print(reference_count[2381731])

In [ ]:
def update_ref_count(row):
    if row['Index_Id'] in reference_count:
        return reference_count[row['Index_Id']]
    else:
        return 0
df["count_reference"] = df.apply(lambda row: update_ref_count(row),axis=1)

In [ ]:
print(df.head(5))

In [ ]:
sorted_by_value = sorted(reference_count.items(), key=lambda kv: kv[1],reverse=True)
print(sorted_by_value[:10])
print([row[0] for row in sorted_by_value[:10]])
print([row[1] for row in sorted_by_value[:10]])

In [ ]:
x_values = [row[0] for row in sorted_by_value[:10]]
y_values = [row[1] for row in sorted_by_value[:10]]
x_pos = np.arange(len(x_values))
plt.figure(figsize=(13,7))
plt.bar(x_pos,y_values, align='center', alpha=1)
plt.xticks(x_pos,x_values)
plt.ylabel('reference count')
plt.title('Top 10 paper by reference')
 
plt.show()

In [ ]:
df["Authors"] = df["Authors"].fillna('')
print(any(df["Authors"].isna()))
df[df['Authors'].str.contains('-')]

In [ ]:
print(df["Authors"].shape)
print(df[df['Authors'].str.isalpha()].shape)

In [ ]:
df[df['Authors'].str.isalpha()]

In [ ]:
df2 = df.filter(regex='spike')

In [135]:
df[df["Index_Id"].isin(inlink[2135000])].shape

(9770, 8)

Above paper is referenced by 9770 other papers

In [137]:
inlink = {}
outlink = {}
#Test for smaller set
testset_len = 10000
for _t, row in df[df["References_Id"].notnull()].iterrows():
    ref = row["References_Id"]
    index = row["Index_Id"]
    ref_list = list(map(int,ref.strip(":").split(":")))
    outlink[index] = ref_list
    for r in ref_list:
        inlist = inlink.get(r,[])
        inlist.append(index)
        inlink[r] = inlist
    #Test for smaller set
    testset_len -= 1
    if not testset_len:
        testset_len = 10000
        break

In [138]:
#Test for smaller set
pagerank = { row["Index_Id"]:1 for _t, row in df[:testset_len].iterrows()}
#complete set
# pagerank = { row["Index_Id"]:1 for _t, row in df.iterrows()}

In [139]:
def create_pagerank(theta, pagerank, inlink, outlink):
    updated_pagerank = {}
    iterations = 0
    while True:
        flag = True
        for index_id, current_score in pagerank.items():
            if index_id in inlink:
                inlink_list = inlink[index_id]
                new_score = 0.0
                for link in inlink_list:
                    len_outlink = len(outlink.get(link, []))
                    if link in pagerank:
                        new_score += pagerank[link]/len_outlink
                new_score = (1.0 - theta) + float(theta)**new_score
                if current_score != new_score:
                    flag = False
                updated_pagerank[index_id] = new_score
        iterations += 1
        if flag == True:
            break
        pagerank = updated_pagerank
        del updated_pagerank
        updated_pagerank = {}
    print(f"number of iterations:{iterations}")
    #normalize the score
    max_score = max(pagerank.values())
    print(f"max score is {max_score}")
    pagerank = { index:score/max_score for index, score in pagerank.items()}

In [140]:
theta = 0.85
create_pagerank(theta, pagerank, inlink, outlink)

number of iterations:14
max score is 1.15


In [92]:
len(set(pagerank.keys()))

10000